# Service status

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
service_status = pd.read_sql_table("mensajeria_estado", OLTP_connection)

In [4]:
pd.read_sql_table("mensajeria_estado", OLTP_connection)

,id,nombre,descripcion
0,4,Recogido por mensajero,Recogido por mensajero
1,5,Entregado en destino,Entregado en destino
2,3,Con novedad,Tiene novedad
3,6,Terminado completo,Terminado completo
4,1,Iniciado,Creado por el usuario
5,2,Con mensajero Asignado,Con mensajero Asignado


## Transformation

In [5]:
service_status = service_status.rename(
	columns={
		"id": "status_id",
		"nombre": "status_name",
		"descripcion": "status_description",
	}
)

In [6]:
service_status = service_status[
	["status_id", "status_name", "status_description"]
]

In [7]:
service_status.head(6)

,status_id,status_name,status_description
0,4,Recogido por mensajero,Recogido por mensajero
1,5,Entregado en destino,Entregado en destino
2,3,Con novedad,Tiene novedad
3,6,Terminado completo,Terminado completo
4,1,Iniciado,Creado por el usuario
5,2,Con mensajero Asignado,Con mensajero Asignado


# Load

In [8]:
service_status.to_sql(
	"SERVICE_STATUS_DIMENSION", OLAP_connection, if_exists="replace", index=False
)

6